In [ ]:
# ========== RADIAL TOKENIZER ==========
import torch
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from radial_tokenizer import RadialTokenizer  # Import from same folder

# ========== CONFIGURATION ==========
IMG_PATH = "C:/Users/denni/Downloads/test.png"  # Update if needed
IMG_NAME = "sample1"
OUTPUT_DIR = "../output"
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D")
ENCODED_DIR = os.path.join(OUTPUT_DIR, "encoded_tokens")
WEIGHTS_DIR = os.path.join(OUTPUT_DIR, "projection_weights")
CSV_DIR = os.path.join(OUTPUT_DIR, "patch_features")
RING_IMG_PATH = os.path.join(OUTPUT_DIR, f"{IMG_NAME}_rings.png")

# Create output folders
os.makedirs(TOK192_DIR, exist_ok=True)
os.makedirs(ENCODED_DIR, exist_ok=True)
os.makedirs(WEIGHTS_DIR, exist_ok=True)
os.makedirs(CSV_DIR, exist_ok=True)

# ========== LOAD & PREPROCESS IMAGE ==========
image = cv2.imread(IMG_PATH)
if image is None:
    raise FileNotFoundError(f"Image not found at {IMG_PATH}")

image = cv2.resize(image, (128, 128))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_norm = image_rgb / 255.0  # Normalize to [0, 1]
img_tensor = torch.from_numpy(image_norm).permute(2, 0, 1).unsqueeze(0).float()  # Shape: [1, 3, 128, 128]

print(f"✅ Loaded Image Tensor Shape: {img_tensor.shape}")

# ========== INITIALIZE TOKENIZER ==========
tokenizer = RadialTokenizer()

# ========== VISUALIZE RINGS ==========
rings = tokenizer.rings
center = tokenizer.center
image_with_rings = image_rgb.copy()

for r0, r1 in rings:
    cv2.circle(image_with_rings, center, r1, (255, 0, 0), 1)  # Outer boundary (blue)
    if r0 > 0:
        cv2.circle(image_with_rings, center, r0, (0, 255, 0), 1)  # Inner boundary (green)

# # Save & show overlay image
# cv2.imwrite(RING_IMG_PATH, cv2.cvtColor(image_with_rings, cv2.COLOR_RGB2BGR))
plt.figure(figsize=(5, 5))
plt.imshow(image_with_rings)
plt.title("Radial Tokenizer Rings Overlay")
plt.axis("off")
plt.show()
# print(f"💾 Saved ring overlay image to: {RING_IMG_PATH}")

# ========== RUN TOKENIZATION ==========
tokens_192d = tokenizer(img_tensor)  # Shape: [1, 4, 192]
print(f"✅ Tokens Shape: {tokens_192d.shape}")

# ========== SAVE REQUIRED OUTPUTS ==========
torch.save(tokens_192d.cpu(), os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))
torch.save(tokenizer.projector.state_dict(), os.path.join(WEIGHTS_DIR, f"{IMG_NAME}.pt"))

# Simulated "encoded" version (if you have encoding later, replace this)
encoded_tokens = torch.mean(tokens_192d, dim=1)
torch.save(encoded_tokens.cpu(), os.path.join(ENCODED_DIR, f"encoded_{IMG_NAME}.pt"))

print(f"💾 Saved 192D tokens to: {os.path.join(TOK192_DIR, f'{IMG_NAME}.pt')}")
print(f"💾 Saved projection weights to: {os.path.join(WEIGHTS_DIR, f'{IMG_NAME}.pt')}")
print(f"💾 Saved encoded tokens to: {os.path.join(ENCODED_DIR, f'encoded_{IMG_NAME}.pt')}")

# ========== INSPECT TOKENS ==========
tokens_np = tokens_192d.squeeze(0).detach().cpu().numpy()  # Shape: [4, 192]

plt.figure(figsize=(10, 5))
for i in range(4):
    plt.plot(tokens_np[i], label=f"Ring {i+1}")
plt.title("192D Projected Tokens Per Ring")
plt.xlabel("Embedding Dimension")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# ========== CSV EXPORT ==========
flattened = tokens_np.flatten()  # Shape: [4*192]
np.savetxt(os.path.join(CSV_DIR, "patch_features.csv"), flattened[None], delimiter=",", fmt="%.6f")
print(f"💾 Saved flattened features to: {os.path.join(CSV_DIR, 'patch_features.csv')}")